# NDVI fourier -> ag

Sources:
- https://www.scielo.br/pdf/pab/v47n9/12.pdf

## Setup

In [ ]:
from pathlib import Path
import pickle

import numpy as np
import rasterio as rio
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans

In [ ]:
# dir_in = Path("../data/ndvi-weekly/")
dir_in = Path("../data/ndvi-modis-daily/")
dir_out = dir_in / "../ndvi-proc/"

dir_in = Path("~/Stanford/data/data/ndvi-daily").expanduser()
dir_out = dir_in.parents[0] / "ndvi-proc"
dir_out.mkdir(exist_ok=True)

tifs = [f for f in dir_in.iterdir() if f.suffix == ".tif"]
print(len(tifs))

In [ ]:
with rio.open(tifs[0]) as ds:
    prof = ds.profile

## Read arrs, stack and FFT

In [ ]:
arrs = []
for f in tifs:
    with rio.open(f) as ds:
        arrs.append(ds.read(1))

In [ ]:
st = np.stack(arrs)
st.shape

In [ ]:
# with open("stack-daily.pickle", "wb") as f:
#     pickle.dump(st, f)

In [ ]:
# with open("stack-daily.pickle", "rb") as f:
#     st = pickle.load(f)

In [ ]:
st.shape, st.dtype

In [ ]:
def fillnan(arr):
    c = 0
    nz = 1
    while nz > 0:
        for i in range(arr.shape[0] - 1):
            mask = np.isnan(arr[i, :, :])
            arr[i, :, :][mask] = arr[i + 1, :, :][mask]
        for i in range(arr.shape[0] - 1):
            ii = 360 - i
            mask = np.isnan(arr[ii, :, :])
            arr[ii, :, :][mask] = arr[ii - 1, :, :][mask]
        nz = np.count_nonzero(np.isnan(arr[:-1, :, :]))
        c += 1
        print(c, nz)
    return arr[:-1, :, :]

In [ ]:
# use abs to convert to real numbers
st = np.abs(np.fft.fft(st, axis=0))
st.shape

In [ ]:
bands = 6

In [ ]:
ft = st[:bands, :, :]
ft = ft.astype(np.float32)
ft.shape

In [ ]:
# with open("fft-daily.pickle", "wb") as f:
#     pickle.dump(ft, f)

In [ ]:
# with open("fft-daily.pickle", "rb") as f:
#     ft = pickle.load(f)

In [ ]:
ft.shape, ft.dtype

In [ ]:
prof.update(count=bands)
with rio.open(dir_out / "daily-fft.tif", "w", **prof) as ds:
    ds.write(ft.astype(np.float32))

# Now let's do ML
- https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans

## KMeans
How many clusters?

In [ ]:
ft2 = np.moveaxis(ft, 0, -1)
ft2.shape

In [ ]:
X = ft2.reshape((-1, bands))
X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
X.shape

In [ ]:
n_clusters = 5
y = KMeans(n_clusters=n_clusters).fit_predict(X)
y.shape

In [ ]:
y_out = y.reshape(st.shape[1:])
y_out.shape

In [ ]:
plt.imshow(y_out)

In [ ]:
prof.update(count=1)
with rio.open(dir_out / "daily-pred.tif", "w", **prof) as ds:
    ds.write(y_out.astype(np.float32), indexes=1)